In [33]:
import pandas as pd

In [34]:
conocimientos_info = pd.read_excel("../data/CUOC/detalle_conocimientos_cuoc_2022.xlsx",sheet_name="conocimientos_cuoc")

In [35]:
ocupaciones_conocimientos = pd.read_excel("../data/CUOC/detalle_conocimientos_cuoc_2022.xlsx",sheet_name="relacion_conocimientos_ocupaci")

In [36]:
conocimientos_info.head()

,Nombre,Descripcion,Codigo
0,Acuicultura y Pesca,"Conocimiento de la cría, el cultivo, y la reco...",83
1,Adquisición del lenguaje,Conocimiento de la estructura y la composición...,52
2,Anatomía,Conocimiento de la estructura del cuerpo humano.,40
3,Armonía y Composición,"Conocimiento de técnicas para la construcción,...",46
4,Arquitectura y Urbanismo,"Conocimiento del arte, la ciencia y las técnic...",53


In [37]:
ocupaciones_conocimientos.head()

,Gran Grupo,Ocupación,Nombre Ocupación,ID Conocimiento,Nombre Conocimiento
0,0,1100,Oficiales de las fuerzas militares,1,Gerencia y administración
1,0,1100,Oficiales de las fuerzas militares,22,Educación y capacitación
2,0,1100,Oficiales de las fuerzas militares,27,Seguridad pública
3,0,1100,Oficiales de las fuerzas militares,28,Leyes y gobierno
4,0,1100,Oficiales de las fuerzas militares,50,Servicio a la persona y humanización


In [38]:
data = pd.merge(
    ocupaciones_conocimientos,
    conocimientos_info,
    left_on = "ID Conocimiento",
    right_on = "Codigo",
    how = "outer"
)

In [39]:
data.isna().sum()

Gran Grupo             1
Ocupación              1
Nombre Ocupación       1
ID Conocimiento        1
Nombre Conocimiento    1
Nombre                 0
Descripcion            0
Codigo                 0
dtype: int64

In [40]:
import pandas as pd
import json


# Agrupamos por 'Codigo'
result = []
for codigo, group in data.groupby('Codigo'):
    title = group['Nombre'].iloc[0]
    description = group['Descripcion'].iloc[0]

    # Ocupaciones únicas (nombre y código)
    occupationTitles = group['Nombre Ocupación'].dropna().unique().tolist()
    occupationUid = group['Ocupación'].dropna().unique().tolist()


    degree = len(occupationUid)

    result.append({
        'Codigo': codigo,
        'title': title,
        'description': description,
        'Ocupaciones_Nombres': occupationTitles,
        'Ocupaciones_Codigos': occupationUid,
        'degree': degree
    })

# Convertimos a JSON
json_result = json.dumps(result, indent=2, ensure_ascii=False)

# Si quieres guardarlo en un archivo:
with open('../output/conocimientos_cuoc.json', 'w', encoding='utf-8') as f:
    f.write(json_result)


In [42]:
import json
import csv
from collections import defaultdict

# Load enriched knowledge data
with open("../output/conocimientos_cuoc.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 1️⃣ Build node list
nodes = []
codigo_to_occupations = {}      # Codigo → set of occupation codes (uids)
codigo_to_titles = {}           # Codigo → {occupation code → occupation title}


for item in data:
    
    codigo = item["Codigo"] #codigo del conocimiento
    title = item["title"]
    description = item["description"]
    occupation_uids = item["Ocupaciones_Codigos"]
    occupation_titles = item["Ocupaciones_Nombres"]
    

    # Add node
    nodes.append({
        "id": codigo,
        "title": title,
        "description": description,
        "occupation_titles": "; ".join(occupation_titles),
        "occupation_uids": "; ".join(map(str,map(int, occupation_uids))),
        "degree": item["degree"]
    })
    
    # Map occupations
    codigo_to_occupations[codigo] = set(map(int, occupation_uids))
    codigo_to_titles[codigo] = dict(zip(map(int, occupation_uids), occupation_titles))

# 2️⃣ Map: occupation code → set of knowledge codes
occ_to_know = defaultdict(set)

for codigo, occupations in codigo_to_occupations.items():
    for occ_uid in occupations:
        occ_to_know[occ_uid].add(codigo)

# 3️⃣ Build weighted edges with shared occupations
edge_weights = defaultdict(int)
edge_shared_occupations = defaultdict(set)

for occ_uid, codigos in occ_to_know.items():
    uris = list(codigos)
    for i in range(len(uris)):
        for j in range(i + 1, len(uris)):
            source, target = sorted([uris[i], uris[j]])
            edge_weights[(source, target)] += 1
            edge_shared_occupations[(source, target)].add(occ_uid)

# 4️⃣ Write nodes.csv
with open("../output/nodes_knowledge_cuoc.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=nodes[0].keys())
    writer.writeheader()
    writer.writerows(nodes)

# 5️⃣ Write edges.csv with shared occupation titles
with open("../output/edges_knowledge_cuoc.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "source", "target", "weight", "shared_occupations_uids","shared_occupations_titles"
    ])
    writer.writeheader()
    
    for (source, target), weight in edge_weights.items():
        shared_uids = edge_shared_occupations[(source, target)] #devuelve uids de las ocupaciones en comun

        # Recolectar nombres de ocupaciones compartidas
        titles = []

        for uid in shared_uids:
            #uid de una ocupacion en comun
            #uid = int(uid)
            title = codigo_to_titles.get(source, {}).get(uid)
            if title:
                titles.append(title)

        writer.writerow({
            "source": source,
            "target": target,
            "weight": weight,
            "shared_occupations_uids": "; ".join(sorted(set(map(str, map(int, shared_uids))))),
            "shared_occupations_titles": "; ".join(set(titles))
        })

print("✅ nodes_knowledge_cuoc.csv and edges_knowledge_cuoc.csv created with weights and shared occupations.")


✅ nodes_knowledge_cuoc.csv and edges_knowledge_cuoc.csv created with weights and shared occupations.
